<a href="https://colab.research.google.com/github/janinewhite/Personal-Expense-Tracker/blob/main/PersonalExpenseTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import csv
import os
import re

class ExpenseTracker:
    def __init__(self, expenses_file: str = "expenses.csv", budget_file: str = "budget.txt"):
        """
        Initializes the ExpenseTracker with specified file names for expenses and budget.

        Args:
            expenses_file (str): The name of the CSV file to store expenses.
            budget_file (str): The name of the TXT file to store the budget.
        """
        self.expenses_file: str = expenses_file
        self.budget_file: str = budget_file
        self.expenses: list[dict] = []
        self.budget: float = 0.0
        self.loaded_files: bool = False # Flag to track if files have been loaded
        self.load_files()
        self.welcome_message_displayed: bool = False # Flag to track if welcome message has been displayed


    def load_csv(self) -> None:
        """
        Loads expenses from a CSV file into the class's expenses list.
        Handles cases where the file doesn't exist or contains invalid entries.
        """
        if os.path.exists(self.expenses_file):
            with open(self.expenses_file, mode='r', newline='', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    try:
                        self.expenses.append({
                            'date': row['date'],
                            'category': row['category'],
                            'amount': float(row['amount']),
                            'description': row['description']
                        })
                    except (ValueError, KeyError):
                        print("⚠️ Skipping invalid entry in expenses file.")
            print(f"ℹ️ Expenses file '{self.expenses_file}' loaded.")
        else:
            print(f"ℹ️ Expenses file '{self.expenses_file}' not found. Starting with an empty expense list.")

    def load_txt(self) -> None:
        """
        Loads the budget from a TXT file into the class's budget variable.
        Handles cases where the file doesn't exist or contains invalid budget format.
        """
        if os.path.exists(self.budget_file):
            with open(self.budget_file, mode='r', encoding='utf-8') as file:
                try:
                    self.budget = float(file.read())
                    print(f"💲 Budget file '{self.budget_file}' loaded.")
                except ValueError:
                    print("⚠️ Invalid budget format in budget file. Budget not loaded.")
        else:
            print(f"ℹ️ Budget file '{self.budget_file}' not found. Starting with a budget of $0.00.")
            self.budget = 0.0

    def load_files(self) -> None:
        """
        Loads expenses and budget from their respective files.
        """
        self.load_csv()
        self.load_txt()
        self.loaded_files = True


    def save_expenses(self) -> None:
        """
        Saves the current list of expenses to a CSV file.
        """
        with open(self.expenses_file, mode='w', newline='', encoding='utf-8') as file:
            fieldnames: list[str] = ['date', 'category', 'amount', 'description']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for expense in self.expenses:
                writer.writerow(expense)
        print("\n✅ Expenses saved.")

    def save_budget(self) -> None:
        """
        Saves the current budget to a TXT file.
        """
        with open(self.budget_file, mode='w', encoding='utf-8') as file:
            file.write(str(self.budget))
        print("$ Budget saved.")

    def save_files(self) -> None:
        """
        Saves the current expenses and budget to their respective files.
        """
        self.save_expenses()
        self.save_budget()

    def add_expense(self) -> None:
        """
        Prompts the user to enter details for a new expense and adds it to the class's expenses list.
        Includes validation for date, category, and amount.
        """
        print("\n📝 Enter expense details:")
        while True:
            date: str = input("Enter date (YYYY-MM-DD): ")
            if re.match(r'\d{4}-\d{2}-\d{2}', date):
                break
            else:
                print("❌ Invalid date format. Please use YYYY-MM-DD.")

        while True:
            category: str = input("Enter category (e.g., Food, Travel): ").strip()
            if category:
                break
            else:
                print("❌ Category cannot be empty. Please enter a category.")

        while True:
            try:
                amount: float = float(input("Enter amount spent: "))
                break
            except ValueError:
                print("❌ Invalid amount. Please enter a valid number.")

        description: str = input("Enter a brief description: ").strip()

        expense: dict = {
            'date': date,
            'category': category,
            'amount': amount,
            'description': description
        }
        self.expenses.append(expense)
        print("✅ Expense added.")

    def view_expenses(self) -> None:
        """
        Display all recorded expenses with validation for required fields.
        """
        if not self.expenses:
            print("📭 No expenses recorded.")
            return

        print("\n📋 Your Expenses:")
        for i, expense in enumerate(self.expenses, start=1):
            required_fields: list[str] = ['date', 'category', 'amount', 'description']
            if all(field in expense for field in required_fields):
                print(f"{i}. {expense['date']} | {expense['category']} | "
                      f"${expense['amount']:.2f} | {expense['description']}")
            else:
                print(f"{i}. ⚠️ Incomplete expense entry.")

    def set_budget(self) -> None:
        """
        Prompts the user to enter their monthly budget and updates the class's budget variable.
        Handles invalid input.
        """
        try:
            self.budget = float(input("\nEnter your monthly budget: "))
            print(f"✅ Budget set to ${self.budget:.2f}")
            self.save_budget()
        except ValueError:
            print("❌ Invalid budget amount.")

    def track_budget(self) -> None:
        """
        Calculates and displays the total amount spent and compares it to the budget.
        """
        total_spent: float = sum(exp['amount'] for exp in self.expenses)
        print(f"\n💰 Total spent: ${total_spent:.2f}")

        if self.budget == 0:
            print("⚠️ No budget set.")
        elif total_spent > self.budget:
            print(f"🚨 You have exceeded your ${self.budget:.2f} budget by ${total_spent - self.budget:.2f} !")
        else:
            print(f"🟢 You have ${self.budget - total_spent:.2f} of your ${self.budget:.2f} budget left for the month.")

    def delete_expense(self) -> None:
        """
        Displays expenses and allows the user to delete one by index.
        Saves expenses after deletion and handles invalid input/selection.
        """
        print("\n🗂️ Select an expense to delete:")
        for i, expense in enumerate(self.expenses, start=1):
            print(f"{i}. {expense['date']} | {expense['category']} | ${expense['amount']:.2f} | {expense['description']}")
        try:
            index: int = int(input("Enter the number of the expense to delete: "))
            if 1 <= index <= len(self.expenses):
                removed: dict = self.expenses.pop(index - 1)
                print(f"✅ Deleted: {removed['date']} | {removed['category']} | ${removed['amount']:.2f}")
                self.view_expenses()
            else:
                print("❌ Invalid selection.")
        except ValueError:
            print("❌ Please enter a valid number.")

    def view_monthly_spend(self) -> None:
        """
        Calculates and displays the total spending for each month,
        indicating whether the amount for the month is under, equal, or over budget.
        """
        if not self.expenses:
            print("📭 No expenses recorded.")
            return

        monthly_spend: dict[str, float] = {}
        for expense in self.expenses:
            try:
                # Extract year and month from the date
                year_month: str = expense['date'][:7]  # Assumes YYYY-MM-DD format
                amount: float = expense['amount']
                if year_month in monthly_spend:
                    monthly_spend[year_month] += amount
                else:
                    monthly_spend[year_month] = amount
            except (KeyError, ValueError):
                print(f"⚠️ Skipping entry with invalid date or amount: {expense}")


        print("\n📈 Monthly Spending:")
        # Sort months chronologically
        for month in sorted(monthly_spend.keys()):
            spend_amount = monthly_spend[month]
            status_icon = ""
            status_text = ""
            if self.budget > 0:
                if spend_amount < self.budget:
                    status_icon = "✅"
                    status_text = "Within Budget"
                elif spend_amount == self.budget:
                    status_icon = "🤝"
                    status_text = "On Budget"
                else:
                    status_icon = "🚨"
                    status_text = "Over Budget"
            print(f"{month}: ${spend_amount:.2f} ({status_icon} {status_text})")


    def print_menu(self, header: str, menu_options: dict) -> None:
        """
        Display menu options with a header.

        Args:
            header (str): The header for the menu.
            menu_options (dict): A dictionary where keys are option numbers and values are tuples
                                 of (description, function).
        """
        print(f"\n{header}:")
        for key, (description, _) in menu_options.items():
            print(f"{key}. {description}")

    def budget_menu(self) -> None:
        """
        Display and handle the budget management menu.
        """
        self.track_budget()
        menu_options: dict = {
            '1': ('Set budget', self.set_budget),
            '2': ('View monthly spend', self.view_monthly_spend),
            '3': ('Return to main menu', self.menu)
        }
        self.print_menu("📌 Budget Menu:", menu_options)
        choice: str = input(f"Choose an option (1-{len(menu_options)}): ")

        if choice in menu_options:
            _, function = menu_options[choice]
            function()
        else:
            print("❌ Invalid choice. Please try again.")
            self.budget_menu()


    def menu(self) -> None:
        """
        Display an interactive menu for the expense tracker application and handle user choices.
        """
        if not self.welcome_message_displayed:
            print("\n👋 Welcome to your personal expense tracker!")
            self.welcome_message_displayed = True

        menu_options: dict = {
            '1': ('Add expense', self.add_expense),
            '2': ('View expenses', self.view_expenses),
            '3': ('Track budget', self.budget_menu),
            '4': ('Save expenses and budget', self.save_files),
            '5': ('Save expenses and budget then exit', self.save_files),
            '6': ('Exit (no save)', None)
        }
        while True:
            self.print_menu("📌 Expenses Menu", menu_options)
            choice: str = input(f"Choose an option (1-{len(menu_options)}): ")
            if choice in menu_options:
                if choice != '6':
                    _, function = menu_options[choice]
                    function()
                if choice == '5' or choice == '6':
                    print("👋 Goodbye!")
                    break
            else:
                print("❌ Invalid choice. Please try again.")

In [ ]:
# Create an instance of the ExpenseTracker class
tracker = ExpenseTracker()
# Start the interactive menu
tracker.menu()

ℹ️ Expenses file 'expenses.csv' loaded.
💲 Budget file 'budget.txt' loaded.

👋 Welcome to your personal expense tracker!

📌 Expenses Menu:
1. Add expense
2. View expenses
3. Track budget
4. Save expenses and budget
5. Save expenses and budget then exit
6. Exit (no save)
Choose an option (1-6): v
❌ Invalid choice. Please try again.

📌 Expenses Menu:
1. Add expense
2. View expenses
3. Track budget
4. Save expenses and budget
5. Save expenses and budget then exit
6. Exit (no save)
Choose an option (1-6): 2

📋 Your Expenses:
1. 2025-10-10 | Food | $15.00 | burger

📌 Expenses Menu:
1. Add expense
2. View expenses
3. Track budget
4. Save expenses and budget
5. Save expenses and budget then exit
6. Exit (no save)
Choose an option (1-6): 3

💰 Total spent: $15.00
🟢 You have $85.00 of your $100.00 budget left for the month.

📌 Budget Menu::
1. Set budget
2. View monthly spend
3. Return to main menu
